In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
# check for correlations
df.corr()

In [ ]:
df = df.dropna(subset = ['HomePlanet', 'CryoSleep', 'VIP', 'Destination', 'Name', 'Cabin', 'VIP'])
df = df.fillna(df.mean(numeric_only = True))
df.info()

In [ ]:
def deckclass(a):
    lst = a.split('/')
    return lst[0]
df['deck'] = df.apply(lambda x: deckclass(x['Cabin']), axis = 1)

def cabinnum(a):
    lst = a.split('/')
    return lst[1]
df['cabin_num'] = df.apply(lambda x: cabinnum(x['Cabin']), axis = 1)

def side(a):
    lst = a.split('/')
    return lst[2]
df['cabin_side'] = df.apply(lambda x: side(x['Cabin']), axis = 1)

df.drop('Cabin', axis = 1, inplace = True)

df.info()

In [ ]:
x = df.drop(['Transported', 'PassengerId', 'Name'], axis = 1)
x = x.replace({'CryoSleep': {True: 1, False: 0}, 'VIP': {True: 1, False: 0}, 'cabin_side': {'S':1, 'P': 0}})

x = pd.get_dummies(x, columns = ['HomePlanet', 'CryoSleep', 'Destination', 'deck'])
y = df['Transported']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 1)
rf = RandomForestClassifier(n_estimators = 1000, random_state = 1)
rf.fit(x_train, y_train)
rf.score(x_test, y_test)

In [ ]:
df_test = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
df_test = df_test.fillna(df_test.mean(numeric_only = True))
df_test = df_test.fillna(method = 'pad')
df_test.info()

In [ ]:
df_test['deck'] = df_test.apply(lambda x: deckclass(x['Cabin']), axis = 1)

df_test['cabin_num'] = df_test.apply(lambda x: cabinnum(x['Cabin']), axis = 1)

df_test['cabin_side'] = df_test.apply(lambda x: side(x['Cabin']), axis = 1)

df_test.drop('Cabin', axis = 1, inplace = True)

pred_x = df_test.drop(['PassengerId', 'Name'], axis = 1)
pred_x = pred_x.replace({'CryoSleep': {True: 1, False: 0}, 'VIP': {True: 1, False: 0}, 'cabin_side': {'S':1, 'P': 0}})

pred_x = pd.get_dummies(pred_x, columns = ['HomePlanet', 'CryoSleep', 'Destination', 'deck'])

df_test['Transported'] = rf.predict(pred_x)
df_test.info()

In [ ]:
result = df_test[['PassengerId', 'Transported']]
result.to_csv('prediction_result.csv', index = False)